In [11]:
pip install -r ../../requirements.txt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


NOTE: Currently, the circuit is too huge, we would do it by requesting covariance and stdev and just calculate it on verifier side instead! Other circuits in this example can run well!

In [12]:
import ezkl
import torch
from torch import nn
import json
import os
import time
import scipy
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math

In [13]:
%run -i ../../zkstats/core.py

In [14]:
# init path
os.makedirs(os.path.dirname('shared/'), exist_ok=True)
os.makedirs(os.path.dirname('prover/'), exist_ok=True)
verifier_model_path = os.path.join('shared/verifier.onnx')
prover_model_path = os.path.join('prover/prover.onnx')
verifier_compiled_model_path = os.path.join('shared/verifier.compiled')
prover_compiled_model_path = os.path.join('prover/prover.compiled')
pk_path = os.path.join('shared/test.pk')
vk_path = os.path.join('shared/test.vk')
proof_path = os.path.join('shared/test.pf')
settings_path = os.path.join('shared/settings.json')
srs_path = os.path.join('shared/kzg.srs')
witness_path = os.path.join('prover/witness.json')
# this is private to prover since it contains actual data
sel_data_path = os.path.join('prover/sel_data.json')
# this is just dummy random value
sel_dummy_data_path = os.path.join('shared/sel_dummy_data.json')

=======================  ZK-STATS FLOW =======================

This example is not necessary. In fact, a person can just request cov(x,y), std(x), and std(y). Then just compute correlation on his own as well, but here we show that the code is composable enough to do all at once. 

In [15]:
data_path = os.path.join('data.json')
dummy_data_path = os.path.join('shared/dummy_data.json')

create_dummy(data_path, dummy_data_path)

In [16]:
scales = [3]
selected_columns = ['x', 'y']
commitment_maps = get_data_commitment_maps(data_path, scales)

In [17]:
# Verifier/ data consumer side: send desired calculation
from zkstats.computation import computation_to_model, State


def computation(s: State, data: list[torch.Tensor]) -> torch.Tensor:
    x, y = data[0], data[1]
    return s.correlation(x, y)

error = 0.01
_, verifier_model = computation_to_model(computation, error)

verifier_define_calculation(dummy_data_path, selected_columns, sel_dummy_data_path, verifier_model, verifier_model_path)

In [18]:
# Prover/ data owner side
_, prover_model = computation_to_model(computation, error)

prover_gen_settings(data_path, selected_columns, sel_data_path, prover_model,prover_model_path, scales, "resources", settings_path)

==== Generate & Calibrate Setting ====
scale:  [3]
setting:  {"run_args":{"tolerance":{"val":0.0,"scale":1.0},"input_scale":3,"param_scale":3,"scale_rebase_multiplier":10,"lookup_range":[-9470,584],"logrows":14,"num_inner_cols":2,"variables":[["batch_size",1]],"input_visibility":{"Hashed":{"hash_is_public":true,"outlets":[]}},"output_visibility":"Public","param_visibility":"Private"},"num_rows":2624,"total_assignments":172,"total_const_size":16,"model_instance_shapes":[[1],[1]],"model_output_scales":[0,3],"model_input_scales":[3,3],"module_sizes":{"kzg":[],"poseidon":[2624,[2]],"elgamal":[0,[0]]},"required_lookups":["Abs",{"GreaterThan":{"a":0.0}}],"check_mode":"UNSAFE","version":"7.0.0","num_blinding_factors":null}


In [19]:
# Here verifier & prover can concurrently call setup since all params are public to get pk.
# Here write as verifier function to emphasize that verifier must calculate its own vk to be sure
setup(verifier_model_path, verifier_compiled_model_path, settings_path,vk_path, pk_path )

print("=======================================")
# Prover generates proof
prover_gen_proof(prover_model_path, sel_data_path, witness_path, prover_compiled_model_path, settings_path, proof_path, pk_path)

==== setting up ezkl ====


spawning module 0
spawning module 2


spawning module 0
spawning module 2


Time setup: 3.3124349117279053 seconds
==== Generating Witness ====
witness boolean:  1.0
witness result 1 : 0.875
==== Generating Proof ====


spawning module 0
spawning module 2


proof:  {'instances': [[[2677402660762584588, 7694868747867669253, 13590833677657947629, 1708151667573093226], [18108060115775388783, 7809885797225827088, 715653157360373353, 3125030121935745365], [12436184717236109307, 3962172157175319849, 7381016538464732718, 1011752739694698287], [3483395353741361115, 3494632259903994625, 6657987792994187913, 108272644256946680]]], 'proof': '1cb9311ee93b1aba1f71dca9497786ebacbac7436c965fecb79125bb1c39ee8125dfaec293e5bab7c26aadb43a1b158bf552aaabb0253483b897e8357d157eeb034b4b9174274573a74e0780478104bc926594286145aed0f536ee601b09e6930a97e62621b063caff104db8d6339e077251a977c9e7c636ade47652a80f4c3629c8f6754fc5a9e3c9907c6b9f2fb34fb0f5941b663d165c719ec0ea52bbc58c202e12193ebcfe3ed69127b0fc08c47f2103834ea8503a9ac27ca0e1a18f0ec30f0f5b10a652fde05684dc2bed5b8c38aed4b3c0acb9124cd6aa9f33e46b8b2c29aee846a49d5daed97cbf04362125c79782eb2bc7b2e20bd6882715d549ac3e058f3e8e4602bd12fbbeaaeb2995f8570f999a4a8c3d76e02820d99a436e9f4a1b95e68eb32bfc4bd9d5fab18307cf58aaea27abc5a

In [20]:
# Verifier verifies
res = verifier_verify(proof_path, settings_path, vk_path, selected_columns, commitment_maps)
print("Verifier gets result:", res)

Verifier gets result: 0.875
